### imports

In [2]:
from sqlalchemy.engine import URL


ModuleNotFoundError: No module named 'sqlalchemy'

### SQLacademy setup

In [ ]:
db_url = URL.create(
    drivername="postgresql+psycopg",
    username="postgres",
    password="password",
    host="localhost",
    port=5432,
    database="similarity_search_service_db"
)